In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import sys
sys.path.append('/content/gdrive/My Drive/Skripsi VMD LSTM Udara/Progress')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt
from torch.utils.data import Dataset, DataLoader
from copy import deepcopy as dc
import math
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler

import torch
import torch.nn as nn
import torch.nn.functional as F

from Modules.LmylFun import LSTM, BLSTM, SimpRNN, NLin, DLin, Lin
from Modules.LmylFun import VMD
from Modules.LmylFun import PostVMD

TabError: inconsistent use of tabs and spaces in indentation (LmylFun.py, line 49)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n_steps = 6
scaler = StandardScaler()

# List Of Training And Test Data Set
trn_li = []
tst_li = []

# Univariate Data List
uni_list = ['banjarbaru_pr',
            'batam_pr',
            'jakarta_pr',
            'jakarta-selatan_pr',
            'jambi_pr',
            'malang_pr',
            'medan_pr',
            'samarinda_pr',
            'semarang_pr']

for fileName in uni_list:
    df = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Univariate/"+fileName+".csv")
    df = df[['Unnamed: 0','pm25']]
    df.drop(df.tail(1).index,inplace=True)

    df_tr = df[[:int(len(df)*(90/100))]]
    df_tr = pd.DataFrame(scaler.fit_transform(df_tr[['pm25']]), columns=['pm25'])

    df_ts = df[[-(int(len(df)*(10/100))):]]
    df_ts = pd.DataFrame(scaler.fit_transform(df_ts[['pm25']]), columns=['pm25'])

    trn_li.append(df_tr)
    tst_li.append(df_ts)

# M4 WEEKLY TRAIN DATA
df_tr = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Multivariate/m4-week-train.csv")
df_tr['V1'] = pd.to_datetime(df_tr['V1'], format='%Y-%m-%d')
df_tr = df_tr[5:]
df_tr = df_tr.set_index('V1')
df_tr = pd.DataFrame(scaler.fit_transform(df_tr[['V2']]), columns=['V2'])

trn_li.append(df_tr)

# M4 WEEKLY TEST DATA
df_ts = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Multivariate/m4-week-test.csv")
df_ts['V1'] = pd.to_datetime(df_ts['V1'], format='%Y-%m-%d')
df_ts = df_ts[:-5]
df_ts = df_ts.set_index('V1')
df_ts = pd.DataFrame(scaler.fit_transform(df_ts[['V2']]), columns=['V2'])

tst_li.append(df_ts)

# SINGAPORE PM10 DATA
df = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Multivariate/central_singapore.csv")
df.drop(df.tail(1).index,inplace=True)
df_tr = df[[:int(len(df)*(90/100))]]
df_tr = pd.DataFrame(scaler.fit_transform(df_tr[['pm10']]), columns=['pm10'])

df_ts = df[[-(int(len(df)*(10/100))):]]
df_ts = pd.DataFrame(scaler.fit_transform(df_ts[['pm10']]), columns=['pm10'])
trn_li.append(df_tr)
tst_li.append(df_ts)

# WIND TURBINE ACTIVEPOWER DATA
df = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Multivariate/Turbine_neo.csv")
df = df[[:-10]]
df_tr = df[[:int(len(df)*(80/100))]]
df_tr = pd.DataFrame(scaler.fit_transform(df_tr[['LV ActivePower (kW)']]), columns=['LV ActivePower (kW)'])

df_ts = df[[-(int(len(df)*(20/100))):]]
df_ts = pd.DataFrame(scaler.fit_transform(df_ts[['LV ActivePower (kW)']]), columns=['LV ActivePower (kW)'])
trn_li.append(df_tr)
tst_li.append(df_ts)

# ETTm2 HUFL DATA
df = pd.read_csv("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/DATASET/Pre-Ready Data/Multivariate/ETTm2_neo.csv")
df = df[[[:-59690]]
df_tr = df[[:int(len(df)*(80/100))]]
df_tr = pd.DataFrame(scaler.fit_transform(df_tr[['HUFL']]), columns=['HUFL'])

df_ts = df[[-(int(len(df)*(20/100))):]]
df_ts = pd.DataFrame(scaler.fit_transform(df_ts[['HUFL']]), columns=['HUFL'])
trn_li.append(df_tr)
tst_li.append(df_ts)

# The Train & Test List Data by Index
# 1. Banjarbaru (PM25)
# 2. Batam (PM25)
# 3. Jakarta (PM25)
# 4. Jakarta-Selatan (PM25)
# 5. Jambi (PM25)
# 6. Malang (PM25)
# 7. Medan (PM25)
# 8. Samarinda (PM25)
# 9. Semarang (PM25)
# 10. M4 Weekly (V1)
# 11. Singapore (PM10)
# 12. WindTurbine (LV ActivePower (kW))
# 13. ETTm2 (HUFL)

In [ ]:
def processResult(out, y_f, y_forsum, sums):
    for i, data in enumerate(out[0]):
        sums.append([])
        for a, dataA in enumerate(out):
            sums[i].append([])

    for i, data in enumerate(y_f[0]):
        y_forsum.append([])
        for a, dataA in enumerate(y_f):
            y_forsum[i].append([])

    #print(sum)
    for i, data in enumerate(out[0]):
        for a, dataA in enumerate(out):
            sums[i][a] = np.array(out[a][i].cpu())
        sums[i] = np.array(sums[i])
        sums[i] = sums[i].sum(axis=0)

    for i, data in enumerate(y_f[0]):
        for a, dataA in enumerate(y_f):
            y_forsum[i][a] = y_f[a][i].tolist()
        #print(y_forsum[i])
        y_forsum[i] = np.array(y_forsum[i])
        y_forsum[i] = y_forsum[i].sum(axis=0)

    for i, d in enumerate(sums):
        sums[i] = d.tolist()
        y_forsum[i] = y_forsum[i].tolist()

    out1 = []
    y1 = []
    #print(len(sum))
    #print(len(y_fsm))
    for i, d in enumerate(sums[0]):
        for o, g in enumerate(sums[0][i]):
            out1.append(g)
            y1.append(y_forsum[i][o])

    #print(sums, '\n')
    #print(y_forsum, '\n')
    #print(len(y1), '\n', y1, '\n\n', len(out1), '\n', out1)
    rmse_final = math.sqrt(mean_squared_error(y1, out1))
    return rmse_final

In [ ]:
def preparing(df, step, textCol):
  df = dc(df)

  for i in range(1, step + 1):
    df[f'pm10(t-{i})'] = df[textCol].shift(i)

  return df

# Class For New DataType
class TimeData(Dataset):
  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return len(self.X)

  def __getitem__(self, i):
    return self.X[i], self.y[i]

# For Model Training And Evaluation
def validate_one_epoch(model, test, lossf_def, show_test):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.train(False)
  running_loss = 0.0

  out_nov = []
  y_fsum = []

  for batch_index, batch in enumerate(test):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    with torch.no_grad():
      output = model(x_batch)
      loss = torch.sqrt(lossf_def(output, y_batch))
      out_nov.append(output)
      y_fsum.append(y_batch)
      #print(x_batch)
      running_loss += loss.item()

  avg_loss_across_batches = running_loss / len(test)

  if show_test:
    print('RMSE Loss: ',avg_loss_across_batches)
    print('---------------------------------------')

  return out_nov, avg_loss_across_batches, y_fsum

def train_one_epoch(model, train, optimz, lossf_def, epoch, max_norm, linear):
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  torch.autograd.set_detect_anomaly(True)
  model.train(True)
  running_loss = 0.0

  for batch_index, batch in enumerate(train):
    x_batch, y_batch = batch[0].to(device), batch[1].to(device)

    output = model(x_batch)

    if output.isnan().any().item() or y_batch.isnan().any().item():
        print(output,' Those are the outputs, and these are the inputs ', x_batch)
        return True

    if linear:
        loss = torch.sqrt(lossf_def(output, y_batch))+0.5*torch.norm(torch.cat([torch.randn(n_steps, 1).view(-1)]), 1)
    else:
        loss = torch.sqrt(lossf_def(output, y_batch))
    running_loss += loss.item()

    optimz.zero_grad()
    loss.backward()

    #torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm)

    optimz.step()

  return False

def train_test_v1(model, train, test, lrn=0.001, iterasi=50, loss=nn.MSELoss(), max_norm=1, show_test=False, linear=False):
  optimz = torch.optim.Adam(model.parameters(), lr=lrn)
  skipTrain = False

  for epoch in range(iterasi):
    if skipTrain == False:
        skipTrain = train_one_epoch(model, train, optimz, loss, epoch, max_norm, linear)
    out, rmse, y_fs = validate_one_epoch(model, test, loss, show_test)

  return out, rmse, y_fs

In [ ]:
def TheVMD(tr, ts, colName, alpha, tau, K, tol, bch, n_step):
    u, u_hat, omega = VMD(tr[colName], alpha, tau, K, 0, 0, tol)
    ut, ut_hat = PostVMD(ts[colName], alpha, tau, K, 0, tol, omega[-1])

    # Preparing Data
    Xs = tr.copy()
    Xts = ts.copy()

    liXs = []
    liXts = []

    for i in range(len(u)):
        Xs[colName] = u[i]
        Xts[colName] = ut[i]
        liXs.append(preparing(Xs, n_step, colName))
        liXts.append(preparing(Xts, n_step, colName))

    for i in range(len(liXs)):
        liXs[i] = liXs[i].drop(index=liXs[i].index[:n_step])
        liXts[i] = liXts[i].drop(index=liXts[i].index[:n_step])

    # Data Splitting
    ys = []
    yts = []

    for a in range(len(liXs)):
        ys.append(liXs[a].pop(colName))
        yts.append(liXts[a].pop(colName))

    # Shifting To Numpy
    liXs_np = []
    liXts_np = []
    ys_np = []
    yts_np = []

    for i in range(len(liXs)):
        liXs_np.append(liXs[i].to_numpy())
        liXts_np.append(liXts[i].to_numpy())
        ys_np.append(ys[i].to_numpy())
        yts_np.append(yts[i].to_numpy())

    # Flipping Train Columns For RNN-Based Algorithm
    for i in range(len(liXs)):
        liXs[i] = dc(np.flip(liXs_np[i], axis=1))
        liXts[i] = dc(np.flip(liXts_np[i], axis=1))

    # Reshaping For Torch
    for i in range(len(liXs)):
        liXs_np[i] = liXs_np[i].reshape((-1, n_step, 1))
        liXts_np[i] = liXts_np[i].reshape((-1, n_step, 1))
        ys_np[i] = ys_np[i].reshape((-1, 1))
        yts_np[i] = yts_np[i].reshape((-1, 1))

    # To TensorTorch
    for i in range(len(liXs)):
        liXs_np[i] = torch.tensor(liXs_np[i]).float()
        liXts_np[i] = torch.tensor(liXts_np[i]).float()
        ys_np[i] = torch.tensor(ys_np[i]).float()
        yts_np[i] = torch.tensor(yts_np[i]).float()

    #print("Train Length " ,len(liXs_np[0]))
    #print("Test Length " ,len(liXts_np[0]))

    Xs_cls = []
    Xts_cls = []
    for i in range(len(liXs_np)):
        Xs_cls.append(TimeData(liXs_np[i], ys_np[i]))
        Xts_cls.append(TimeData(liXts_np[i], yts_np[i]))

    trn_load_vmd = []
    tst_load_vmd = []

    for i in range(len(Xs_cls)):
        trn_load_vmd.append(DataLoader(Xs_cls[i], batch_size=bch, shuffle=True))
        tst_load_vmd.append(DataLoader(Xts_cls[i], batch_size=bch, shuffle=True))

    return trn_load_vmd, tst_load_vmd

In [ ]:
def PredictVMD(tr, ts, batches, n_step, colName):
    trnLSTM, tstLSTM = TheVMD(tr, ts, colName, 5000, 0, 30, 1e-5, batches, n_step)
    trnBLSTM, tstBLSTM = TheVMD(tr, ts, colName, 7000, 1, 20, 1e-7, batches, n_step)
    trnRNN, tstRNN = TheVMD(tr, ts, colName, 7000, 0, 30, 1e-5, batches, n_step)
    trnLin, tstLin = TheVMD(tr, ts, colName, 9000, 0, 10, 1e-7, batches, n_step)
    trnDLin, tstDLin = TheVMD(tr, ts, colName, 9000, 0, 30, 1e-5, batches, n_step)
    trnNLin, tstNLin = TheVMD(tr, ts, colName, 3000, 0, 20, 1e-7, batches, n_step)

    li_lstm = []
    li_blstm = []
    li_rnn = []
    li_lin = []
    li_dlin = []
    li_nlin = []

    # Define The Models
    for modi in range(len(trnLin)):
        lin = Lin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 30
        lin.to(device)
        li_lin.append(lin)

    for magni in range(len(trnDLin)):
        dlin = DLin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 60
        dlin.to(device)
        li_dlin.append(dlin)

        lstm = LSTM(1, 30, 1) # Dont Forget To Set Epoch = 70
        lstm.to(device)
        li_lstm.append(lstm)

        rnn = SimpRNN(1, 40, 1) # Dont Forget To Set Epoch = 30
        rnn.to(device)
        li_rnn.append(rnn)

    for lavender in range(len(trnNLin)):
        nlin = NLin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 70
        nlin.to(device)
        li_nlin.append(nlin)

        blstm = BLSTM(1, 10, 1) # Dont Forget To Set Epoch = 30
        blstm.to(device)
        li_blstm.append(blstm)

    # Start Predicting
    outLin = []
    y_fsmLin = []
    rmse_finLin = []
    rmseLin = 0

    outDLin = []
    y_fsmDLin = []
    rmse_finDLin = []
    rmseDLin = 0

    outNLin = []
    y_fsmNLin = []
    rmse_finNLin = []
    rmseNLin = 0

    out = []
    y_fsm = []
    rmse_fin = []
    rmse = 0

    outBLSTM = []
    y_fsmBLSTM = []
    rmse_finBLSTM = []
    rmseBLSTM = 0

    outRNN = []
    y_fsmRNN = []
    rmse_finRNN = []
    rmseRNN = 0

    for i in range(len(li_lstm)):
        tmp, rmse_tmp, y_f = train_test_v1(li_lstm[i], trnLSTM[i], tstLSTM[i], show_test=False, iterasi=70, linear=True)
        out.append(tmp)
        y_fsm.append(y_f)
        rmse_fin.append(rmse_tmp)
        rmse += rmse_tmp

        tmp, rmse_tmp, y_f = train_test_v1(li_rnn[i], trnRNN[i], tstRNN[i], show_test=False, iterasi=30, linear=True)
        outRNN.append(tmp)
        y_fsmRNN.append(y_f)
        rmse_finRNN.append(rmse_tmp)
        rmseRNN += rmse_tmp

        tmp, rmse_tmp, y_f = train_test_v1(li_dlin[i], trnDLin[i], tstDLin[i], show_test=False, iterasi=60, linear=True)
        outDLin.append(tmp)
        y_fsmDLin.append(y_f)
        rmse_finDLin.append(rmse_tmp)
        rmseDLin += rmse_tmp

    for a in range(len(li_blstm)):
        tmp, rmse_tmp, y_f = train_test_v1(li_nlin[a], trnNLin[a], tstNLin[a], show_test=False, iterasi=70, linear=True)
        outNLin.append(tmp)
        y_fsmNLin.append(y_f)
        rmse_finNLin.append(rmse_tmp)
        rmseNLin += rmse_tmp

        tmp, rmse_tmp, y_f = train_test_v1(li_blstm[a], trnBLSTM[a], tstBLSTM[a], show_test=False, iterasi=30, linear=True)
        outBLSTM.append(tmp)
        y_fsmBLSTM.append(y_f)
        rmse_finBLSTM.append(rmse_tmp)
        rmseBLSTM += rmse_tmp

    for o in range(len(li_lin)):
        tmp, rmse_tmp, y_f = train_test_v1(li_lin[o], trnLin[o], tstLin[o], show_test=False, iterasi=30, linear=True)
        outLin.append(tmp)
        y_fsmLin.append(y_f)
        rmse_finLin.append(rmse_tmp)
        rmseLin += rmse_tmp

    # Process The Result For RNN Based
    sumi = []
    y_forsum = []
    rmseLSTM = processResult(out, y_fsm, y_forsum, sumi)

    sumBLSTM = []
    y_forsumBLSTM = []
    rmseBLSTM = processResult(outBLSTM, y_fsmBLSTM, y_forsumBLSTM, sumBLSTM)

    sumRNN = []
    y_forsumRNN = []
    rmseRNN = processResult(outRNN, y_fsmRNN, y_forsumRNN, sumRNN)

    # Process The Result For Linear Based
    sumLin = []
    y_forsumLin = []
    rmseLin = processResult(outLin, y_fsmLin, y_forsumLin, sumLin)

    sumDLin = []
    y_forsumDLin = []
    rmseDLin = processResult(outDLin, y_fsmDLin, y_forsumDLin, sumDLin)

    sumNLin = []
    y_forsumNLin = []
    rmseNLin = processResult(outNLin, y_fsmNLin, y_forsumNLin, sumNLin)

    return [rmseLSTM, rmseBLSTM, rmseRNN, rmseLin, rmseDLin, rmseNLin]

In [ ]:
def processResultNoVMD(out, y_f, y_forsum, sums):
    '''for i, data in enumerate(out[0]):
        sums.append([])
        for a, dataA in enumerate(out):
            sums[i].append([])

    for i, data in enumerate(y_f[0]):
        y_forsum.append([])
        for a, dataA in enumerate(y_f):
            y_forsum[i].append([])

    #print(sum)
    for i, data in enumerate(out[0]):
        for a, dataA in enumerate(out):
            sums[i][a] = np.array(out[a][i].cpu())
        sums[i] = np.array(sums[i])
        sums[i] = sums[i].sum(axis=0)

    for i, data in enumerate(y_f[0]):
        for a, dataA in enumerate(y_f):
            y_forsum[i][a] = y_f[a][i].tolist()
        #print(y_forsum[i])
        y_forsum[i] = np.array(y_forsum[i])
        y_forsum[i] = y_forsum[i].sum(axis=0)

    for i, d in enumerate(sums):
        sums[i] = d.tolist()
        y_forsum[i] = y_forsum[i].tolist()

    out1 = []
    y1 = []
    #print(len(sum))
    #print(len(y_fsm))
    for i, d in enumerate(sums[0]):
        for o, g in enumerate(sums[0][i]):
            out1.append(g)
            y1.append(y_forsum[i][o])

    #print(sums, '\n')
    #print(y_forsum, '\n')
    #print(len(y1), '\n', y1, '\n\n', len(out1), '\n', out1)'''
    rmse_final = math.sqrt(mean_squared_error(y_f, out))
    return rmse_final

def trainModelNoVMD(train, test, epoch, model):
  out = []
  y_fsm = []
  rmse_fin = []
  rmse = 0

  tmp, rmse_tmp, y_f = train_test_v1(model, train, test, show_test=False, iterasi=epoch, linear=True)
  out.append(tmp)
  y_fsm.append(y_f)
  rmse_fin.append(rmse_tmp)
  rmse += rmse_tmp

  sumi = []
  y_forsum = []
  rmse = processResultNoVMD(out, y_fsm, y_forsum, sumi)

  return rmse

In [ ]:
def PredictNoVMD(tr, ts, batches, n_step, colName):
  lin = Lin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 30
  lin.to(device)

  dlin = DLin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 60
  dlin.to(device)

  nlin = NLin(n_step, batches, 1, individual=True) # Dont Forget To Set Epoch = 70
  nlin.to(device)

  lstm = LSTM(1, 30, 1) # Dont Forget To Set Epoch = 70
  lstm.to(device)

  blstm = BLSTM(1, 10, 1) # Dont Forget To Set Epoch = 30
  blstm.to(device)

  rnn = SimpRNN(1, 40, 1) # Dont Forget To Set Epoch = 30
  rnn.to(device)

  rmseLin = trainModelNoVMD(tr, ts, 30, lin)
  rmseDLin = trainModelNoVMD(tr, ts, 60, dlin)
  rmseNLin = trainModelNoVMD(tr, ts, 70, nlin)

  rmseLSTM = trainModelNoVMD(tr, ts, 70, lstm)
  rmseBLSTM = trainModelNoVMD(tr, ts, 30, blstm)
  rmseRNN = trainModelNoVMD(tr, ts, 30, rnn)

In [ ]:
rmseModel = []

for idx, data in enumerate(trn_li):
    col = list(data.columns.values)[-1]
    trn_li[idx] = pd.DataFrame(trn_li[idx][col])
    tst_li[idx] = pd.DataFrame(tst_li[idx][col])

    tmpModel = PredictVMD(trn_li[idx], tst_li[idx], 6, 6, col)
    print(tmpModel)
    rmseModel.append(tmpModel)

with open("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/Progress/Result/resultUniVMD.pkl", 'wb') as f:
    pickle.dump(rmseModel, f)
    print("File Successfully Created")

In [ ]:
rmseModelNoVMD = []

for idx, data in enumerate(trn_li):
    col = list(data.columns.values)[-1]
    trn_li[idx] = pd.DataFrame(trn_li[idx][col])
    tst_li[idx] = pd.DataFrame(tst_li[idx][col])

    tmpModel = PredictNoVMD(trn_li[idx], tst_li[idx], 6, 6, col)
    print(tmpModel)
    rmseModelNoVMD.append(tmpModel)

with open("/content/gdrive/My Drive/Skripsi VMD LSTM Udara/Progress/Result/resultUni.pkl", 'wb') as f:
    pickle.dump(rmseModelNoVMD, f)
    print("File Successfully Created")